# 设备监控智能Agent使用示例

本Notebook展示如何使用基于Qwen3-32B的设备监控Agent进行：
- 设备状态分析
- 日志分析
- 维修历史分析
- 综合诊断
- 生成行动计划

**前提条件**：
1. 已设置环境变量 `XPULINK_API_KEY`
2. 已安装所需依赖：`pip install requests python-dotenv`

## 1. 初始化和配置

In [ ]:
import os
import json
import sys
from pathlib import Path

# 添加项目路径
sys.path.append(str(Path.cwd()))

from device_agent import DeviceMonitorAgent

# 检查API Key
api_key = os.getenv("XPULINK_API_KEY")
if not api_key:
    print("⚠️  请设置环境变量 XPULINK_API_KEY")
    print("可以使用: os.environ['XPULINK_API_KEY'] = 'your-api-key'")
else:
    print("✓ API Key 已配置")

# 初始化Agent
agent = DeviceMonitorAgent()
print("✓ 设备监控Agent初始化成功")

## 2. 加载示例数据

In [ ]:
# 加载设备数据
with open('data/examples/device_data.json', 'r', encoding='utf-8') as f:
    device_data = json.load(f)

print("设备信息：")
print(f"  设备ID: {device_data['device_id']}")
print(f"  设备名称: {device_data['device_name']}")
print(f"  设备类型: {device_data['device_type']}")
print(f"  运行状态: {device_data['status']['operational_status']}")
print(f"  告警级别: {device_data['status']['alert_level']}")
print(f"\n关键传感器读数：")
print(f"  温度: {device_data['sensors']['temperature']['value']}°C (阈值: {device_data['sensors']['temperature']['threshold_max']}°C)")
print(f"  压力: {device_data['sensors']['pressure']['value']} MPa")
print(f"  振动: {device_data['sensors']['vibration']['value']} mm/s (阈值: {device_data['sensors']['vibration']['threshold_max']} mm/s)")
print(f"  功率: {device_data['sensors']['power_consumption']['value']} kW")

In [ ]:
# 加载日志数据
with open('data/examples/device_logs.txt', 'r', encoding='utf-8') as f:
    logs = [line.strip() for line in f.readlines() if line.strip()]

print(f"✓ 已加载 {len(logs)} 条日志记录")
print("\n最近的5条日志：")
for log in logs[:5]:
    print(f"  {log}")

In [ ]:
# 加载维修记录
with open('data/examples/maintenance_records.json', 'r', encoding='utf-8') as f:
    maintenance_records = json.load(f)

print(f"✓ 已加载 {len(maintenance_records)} 条维修记录")
print("\n最近的3次维修：")
for record in maintenance_records[:3]:
    print(f"\n  日期: {record['date']}")
    print(f"  类型: {record['type']}")
    print(f"  描述: {record['description']}")
    print(f"  成本: ¥{record['total_cost']}")

## 3. 设备状态分析

使用Agent分析当前设备状态，识别潜在问题

In [ ]:
print("🔍 正在分析设备状态...\n")
device_status_result = agent.analyze_device_status(device_data)

print("=" * 80)
print("设备状态分析结果")
print("=" * 80)
print(device_status_result['analysis'])
print("\n" + "=" * 80)

## 4. 日志分析

分析历史日志，识别错误模式和趋势

In [ ]:
print("📊 正在分析日志数据...\n")
log_analysis_result = agent.analyze_logs(
    logs,
    context=f"设备ID: {device_data['device_id']}, 设备类型: {device_data['device_type']}"
)

print("=" * 80)
print("日志分析结果")
print("=" * 80)
print(log_analysis_result['analysis'])
print("\n" + "=" * 80)

## 5. 维修历史分析

分析维修历史，识别高频故障和改进机会

In [ ]:
print("🔧 正在分析维修历史...\n")
maintenance_analysis_result = agent.analyze_maintenance_history(maintenance_records)

print("=" * 80)
print("维修历史分析结果")
print("=" * 80)
print(maintenance_analysis_result['analysis'])
print("\n" + "=" * 80)

## 6. 综合诊断报告

整合所有数据源，生成全面的诊断报告

In [ ]:
print("🎯 正在生成综合诊断报告...\n")
print("这可能需要1-2分钟，请耐心等待...\n")

comprehensive_report = agent.comprehensive_diagnosis(
    device_data=device_data,
    logs=logs[:30],  # 使用最近30条日志
    maintenance_records=maintenance_records
)

print("=" * 80)
print("综合诊断报告")
print("=" * 80)
print(comprehensive_report['report'])
print("\n" + "=" * 80)

## 7. 生成行动计划

基于诊断报告生成具体的行动计划

In [ ]:
print("📋 正在生成行动计划...\n")

action_plan = agent.generate_action_plan(
    diagnosis_report=comprehensive_report['report'],
    priority="all"
)

print("=" * 80)
print("行动计划")
print("=" * 80)
print(action_plan['action_plan'])
print("\n" + "=" * 80)

## 8. 导出报告

将诊断报告保存为文件

In [ ]:
# 导出为Markdown格式
report_path = agent.export_report(
    comprehensive_report,
    output_path="data/reports/diagnosis_report.md",
    format="markdown"
)
print(f"✓ Markdown报告已保存到: {report_path}")

# 导出为JSON格式
json_path = agent.export_report(
    comprehensive_report,
    output_path="data/reports/diagnosis_report.json",
    format="json"
)
print(f"✓ JSON报告已保存到: {json_path}")

## 9. 交互式问答

向Agent提问关于设备的问题

In [ ]:
# 示例问题
questions = [
    "当前设备最需要关注的问题是什么？",
    "温度超标可能的原因有哪些？",
    "根据维修历史，这台设备的薄弱环节在哪里？",
]

context = {
    "device_data": device_data,
    "recent_logs": logs[:10],
    "recent_maintenance": maintenance_records[:3]
}

for i, question in enumerate(questions, 1):
    print(f"\n{'='*80}")
    print(f"问题 {i}: {question}")
    print('='*80)
    
    answer = agent.query(question, context=context)
    print(answer)
    print()

## 10. 自定义分析

你可以根据实际需求自定义分析

In [ ]:
# 示例：针对特定问题的深度分析
custom_question = """
请针对以下观察进行深度分析：
1. 温度持续超过阈值
2. 振动水平升高
3. 功率消耗增加

这三个现象之间可能存在什么关联？可能的根本原因是什么？
请提供详细的技术分析和解决方案。
"""

print("🔬 深度分析中...\n")
deep_analysis = agent.query(custom_question, context=context)

print("=" * 80)
print("深度分析结果")
print("=" * 80)
print(deep_analysis)
print("\n" + "=" * 80)

## 总结

通过本示例，你已经学会了如何：

✅ 初始化和配置设备监控Agent  
✅ 分析设备状态数据  
✅ 分析历史日志  
✅ 分析维修记录  
✅ 生成综合诊断报告  
✅ 生成行动计划  
✅ 导出报告  
✅ 进行交互式问答  

### 下一步

- 使用你自己的设备数据进行分析
- 集成到现有的监控系统
- 定制化Agent的系统提示词
- 构建自动化的监控和告警流程

### 注意事项

⚠️ **API调用成本**: 每次分析都会调用云端API，请注意成本控制  
⚠️ **数据隐私**: 确保敏感数据经过脱敏处理  
⚠️ **结果验证**: AI分析结果应由专业人员验证  